In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import shutil
import glob
import math
import random
from google.colab.patches import cv2_imshow
import tensorflow.image as tfi
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D ,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.applications.mobilenet import MobileNet,preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#!pip install opendatasets


from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import opendatasets as od

from google.colab import files



In [ ]:
od.download("https://www.kaggle.com/datasets/aryashah2k/breast-ultrasound-images-dataset")


In [ ]:
#Remove mask

b="/content/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign"
m="/content/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/malignant"
n="/content/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/normal"
for dir in os.listdir(n):
  
  p=os.path.join(n,dir)
  isexist = os.path.exists(p)
  if p.find("_mask")>=0:
    print(p)
    print(isexist)
    if isexist :
      print(p)
      os.remove(p)
for dir in os.listdir(b):
  
  p=os.path.join(b,dir)
  isexist = os.path.exists(p)
  if p.find("_mask")>=0:
    print(p)
    print(isexist)
    if isexist :
      print(p)
      os.remove(p)
for dir in os.listdir(m):
  
  p=os.path.join(m,dir)
  isexist = os.path.exists(p)
  if p.find("_mask")>=0:
    print(p)
    print(isexist)
    if isexist :
      print(p)
      os.remove(p)



  


In [ ]:

image2 = cv2.imread('/content/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (10).png')
heatmap1 = cv2.applyColorMap(image2, cv2.COLORMAP_JET)
heatmap2 = cv2.cvtColor(heatmap1, cv2.COLOR_BGR2RGB)


cv2.GaussianBlur(image2)
cv2_imshow(heatmap2)
cv2_imshow(image2)


In [ ]:
#uploaded = files.upload()


In [ ]:
#count images according to classes 
pathTrain="/content/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT"
pathCV = "/content/breast-ultrasound-images-dataset/test"
pathTest="/content/breast-ultrasound-images-dataset/cv"
img_height=224
img_width=224
batchSize = 32

numberOfTrain = {}
numberOfCV = {}
numberOfTest={}

images_train_data={}
images_CV_data={}
images_Test_data={}


pathOfEachTrain=[]
pathOfEachCV=[]
pathOfEachtest=[]
labels=[]
n_images=0
for dir in os.listdir(pathTrain):
  images_train_data[dir]=os.listdir(os.path.join(pathTrain,dir))
  pathOfEachTrain.append(os.path.join(pathTrain,dir))
  numberOfTrain[dir] = len(os.listdir(os.path.join(pathTrain,dir)))
  n_images+=len(os.listdir(os.path.join(pathTrain,dir)))
for dir in os.listdir(pathCV):
  images_CV_data[dir]=os.listdir(os.path.join(pathCV,dir))
  pathOfEachCV.append(os.path.join(pathCV,dir))
  numberOfCV[dir] = len(os.listdir(os.path.join(pathCV,dir)))
for dir in os.listdir(pathTest):
  images_Test_data[dir]=os.listdir(os.path.join(pathTest,dir))
  pathOfEachtest.append(os.path.join(pathTest,dir))
  numberOfTest[dir] = len(os.listdir(os.path.join(pathTest,dir)))

def spliting(n_images,dir_name,percent=0.05):
  count=-1
  for i in images_train_data:
    tmp=[] 
    tmp=images_train_data[i]
    tmp=tmp[0:math.ceil(n_images*percent)]
    count+=1
    for j in tmp:
        newpath = f'/content/breast-ultrasound-images-dataset/{dir_name}/{i}'
        if not os.path.exists(newpath):
               os.makedirs(newpath)
        shutil.move(os.path.join(pathOfEachTrain[count],j),newpath)







               
               
  
   
   
#spliting(n_images,'cv',0.05)
#spliting(n_images,'test',0.05)


In [ ]:
n_images
numberOfTrain
labels=['benign','malignant','normal']
labels
images_train_data

In [ ]:
#data augm train
trainGen = ImageDataGenerator(
                             vertical_flip=True,
                             rescale=1/255,
                             #horizontal_flip = True
                              )

    
z=0
for j in images_train_data['normal']:
    img = load_img(os.path.join(pathTrain,'normal',j))
    x=img_to_array(img)
    x=x.reshape((1,)+x.shape)
    count=0
    z+=1
    for batch in trainGen.flow(x,save_to_dir=f"{pathTrain}/normal",save_prefix=f'normal_generated{z}',save_format='png'):
        count+=1
        if count>1:
           break

     
Train_prepare = trainGen.flow_from_directory(directory = pathTrain,target_size=(224,224),batch_size=32,class_mode='categorical')


In [ ]:
test_prepare = trainGen.flow_from_directory(directory = pathTest,target_size=(224,224),batch_size=7,class_mode='categorical')


In [ ]:
CV_prepare = trainGen.flow_from_directory(directory = pathTest,target_size=(224,224),batch_size=7,class_mode='categorical')


In [ ]:
# load the InceptionResNetV2 architecture with imagenet weights as base
base_model = tf.keras.applications.InceptionResNetV2(
					include_top=False,
					weights='imagenet',
					input_shape=(224,224,3)
					)

base_model.trainable=False
# For freezing the layer we make use of layer.trainable = False
# means that its internal state will not change during training.
# model's trainable weights will not be updated during fit(),
# and also its state updates will not run.

model = tf.keras.Sequential([
		base_model,
		tf.keras.layers.BatchNormalization(renorm=True),
		tf.keras.layers.GlobalAveragePooling2D(),
		tf.keras.layers.Dense(512, activation='relu'),
		tf.keras.layers.Dense(256, activation='relu'),
		tf.keras.layers.Dropout(0.5),
		tf.keras.layers.Dense(128, activation='relu'),
		tf.keras.layers.Dense(3, activation='softmax')
	])


In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
es = EarlyStopping(monitor="val_accuracy",min_delta=0.01,patience=3,verbose=1,mode='auto')
mc = ModelCheckpoint(monitor="val_accuracy",filepath="mymodel.h5",save_best_only=True,verbose=1,mode='auto') 
cd=[es,mc] 

In [ ]:
history = model.fit_generator(
  Train_prepare,
  validation_data=CV_prepare,
  epochs=30,
  callbacks=cd
)

In [ ]:
f=history.history
print(f.keys())
print(max(f['accuracy']))
print(max(f['val_accuracy']))
print(min(f['loss']))
print(min(f['val_loss']))

In [ ]:
plt.plot(f['loss'])
plt.plot(f['val_loss'],c="red")
plt.title("model beh")
plt.show()

In [ ]:
from keras.models import load_model
model = load_model("/content/mymodel.h5")
     

In [ ]:
accuracy_score = model.evaluate(test_prepare)
print(accuracy_score)
print("Accuracy: {:.4f}%".format(accuracy_score[1] * 100))
 
print("Loss: ",accuracy_score[0])

In [ ]:
path = "/content/A318601_1_En_3_Fig3_HTML.jpg"
img = load_img(path,target_size=(224,224))
input_arr = img_to_array(img)/255
plt.imshow(input_arr)
plt.show()
input_arr=np.expand_dims(input_arr,axis=0)
pred=model.predict(input_arr)
score =pred[0]
print(labels[np.argmax(score)])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(labels[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
path = "/content/breast-ultrasound-images-dataset/test/benign/benign (12).png"
img = load_img(path,target_size=(224,224))
input_arr = img_to_array(img)/255
plt.imshow(input_arr)
plt.show()
input_arr=np.expand_dims(input_arr,axis=0)
pred=model.predict(input_arr)
score =pred[0]
print(labels[np.argmax(score)])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(labels[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
path = "/content/breast-ultrasound-images-dataset/test/malignant/malignant (48).png"
img = load_img(path,target_size=(224,224))
input_arr = img_to_array(img)/255
plt.imshow(input_arr)
plt.show()
input_arr=np.expand_dims(input_arr,axis=0)
pred=model.predict(input_arr)
score =pred[0]
print(labels[np.argmax(score)])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(labels[np.argmax(score)], 100 * np.max(score))
)